In [480]:
!pip install chess
!pip install stockfish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [481]:
import chess
import random
random.seed(10)
#from stockfish import Stockfish

In [482]:
indices = ['P','R','N','B','Q','K']
indices += [l.lower() for l in indices]
# por indexado de listas en python => indice-1
indices = {v:k for k,v in enumerate(indices)}

In [483]:
#The most commonly used bitstring length for chess is 64 bits
def rand_bitstring(l=64):
  return random.randint(1,(2**l)-1)

In [484]:
def zoibrist_hash(table, board):
  global indices
  h = 0
  h = h ^ table.black_not_castle
  h = h ^ table.black_not_castle
  if board.turn == chess.BLACK:
    h = h ^ table.black_to_move
  for i in range(64):
    p = board.piece_at(i)
    if p != None:
      h = h ^ table.tab[i][indices[p.symbol()]]
  return h

In [485]:
class BestMove():

  def __init__(self,move,val,depth):
    self.move  = move
    self.val   = val
    self.depth = depth
  
  def __str__(self):
    return str(self.move) + ' - ' + str(self.val) + ' - ' + str(self.depth) 


In [486]:
# create a table for every board traversed during minimax_ab
class Tabla():
  def __init__(self):
    self.tab = [[rand_bitstring() for j in range(12)] for _ in range(64)] 
    self.black_to_move = 0
    # usar los siguientes campos
    self.black_not_castle = 0
    self.white_not_castle = 0
    # fill the best-move during minimax_ab execution
    self.best_move = BestMove(None,None,None)

In [487]:

class Engine():


    # constructor
    def __init__(self,depth=3) -> None:
        #self.eng = Stockfish()
        self.depth   = depth
        self.posCont = 0
        self.tabla   = Tabla()
        self.d       = {}


    # get_material_v -> devuelve en un valor numérico el desequilibrio material
    def get_material_v(self, board):
        v = 0
        for i in range(64):
            piece = board.piece_at(i)
            if piece:
                piece_value = chess.PIECE_TYPES[piece.piece_type-1]
                if piece.color == chess.WHITE:
                    v += piece_value
                else:
                    v -= piece_value
        return v


    # heuristic_value -> devuelve un valor heurístico para un nodo
    def heuristic_value(self, board):
        # tablero terminal
        if board.is_game_over():
            res = board.outcome().result()
            if '/' in res:
                return 0
            else:
                return 100_000 if res[0] == '1' else -100_000
        # evaluación heurística de material
        else:
            return self.get_material_v(board)


    # rate_move -> devuelve un valor de cuán bueno es un movimiento 'a primeras'
    def rate_move(self, board, move):

        # devolvemos valor negativo
        if not board.is_valid() or move == None or move not in board.legal_moves:
            return -100
        
        # guardamos el valor antes del movimiento
        v = 0.
        ov = self.heuristic_value(board)
        
        # hacemos el movimiento
        board.push(move)

        # aplicar heurística de jaques 
        if board.is_check():
            v += 1.5
        # aplicar heurística de material
        nv              = self.heuristic_value(board)
        piece_moved     = board.piece_at(move.from_square)
        if piece_moved:
            piece_moved_val = chess.PIECE_TYPES[piece_moved.piece_type-1]
            if nv > ov:
                # TODO: REFACTOR - PSEUDO MVP-LVP
                # cuanto más vale la pieza con la que capturamos, menos vale la captura
                v += abs(nv-ov) * ( 1 / piece_moved_val )
            elif nv < ov:
                v -= abs(nv-ov) * ( 1 / piece_moved_val )

        # deshacemos el movimiento y devolvemos el valor 'a pripri'
        board.pop()
        return v


    def get_childs(self, board):
        to_explore = list(board.legal_moves) 
        # no te olvides del reverse = True, melón
        to_explore.sort(key=lambda x: self.rate_move(board,x),reverse=True)
        moves = []
        hash = zoibrist_hash(self.tabla,board)
        if hash in self.d.keys() and self.d[hash] != None:
          moves.append(self.d[hash].move)
        
        for m in to_explore:
            board.push(m)
            if board.is_valid():
                moves.append(m)
            board.pop()
        return moves



    # alfa-beta prunning + memoization - LONG + "FAST" VERSION
    def minimax_ab(self, board, depth, maximize, alfa, beta,moves=[]):
      
        self.posCont += 1
        
        if depth == 0 or board.is_game_over():
            return self.heuristic_value(board)

        hash = zoibrist_hash(self.tabla, board)
        #actual_move = None
        if maximize:
            # estamos maximizando - actualizamos alfa
            act = -100_000
            anterior = None
            for f in self.get_childs(board):
                board.push(f)
                act2 = act -1 +1
                act = max(act,self.minimax_ab(board, depth-1, False, alfa, beta))
                if anterior == None or act > act2:
                  anterior = f
                  self.d[hash] = BestMove(anterior,act,depth)
                board.pop()
                # cortamos cuando nuestro valor_a_maximizar sea mayor que el del minimizador
                # el adversario (mimizador) siempre escogera el menor en su turno
                if act >= beta:
                    break
                alfa = max(alfa, act)
            return act
        else:
            # estamos minimizando - actualizamos alfa
            act = 100_000
            for f in self.get_childs(board):
                board.push(f)
                act = min(act,self.minimax_ab(board, depth-1, True, alfa, beta))
                # cortamos cuando nuestro valor_a_minimizar sea menor que el del adversario
                # él como maximizador siempre escogera el mayor de los valores en su turno
                board.pop()
                if act <= alfa:
                    break
                beta = min(beta, act)
        """
        if actual_move != None and (hash not in self.d.keys() or self.d[hash].depth < depth):
          self.d[hash] = actual_move
        """
        return act



    def get_move(self, board):

        # TODO: IMPLEMENTAR estructura <GAMETREE> (para memoization) - almacenar los nodos recorridos y sus valores
        #   - ir recalculando recursivamente las últimas hojas candidatas según aumentemos la profundidad
        # TABLA DE TRANSPOSICIONES CON NOTACIÓN ZOIBRIST
        hash = zoibrist_hash(self.tabla,board)

        if not hash in self.d.keys() or self.d[hash].depth < self.depth:
          maximize = board.turn == chess.WHITE
          best_v = -100_000 if maximize else 100_000
          actual_move, best_m = None, None
          # recorremos hasta 20 primeros hijos del nodo actual
          for m in self.get_childs(board):
              board.push(m)
              val = self.minimax_ab(
                  board, self.depth, not maximize, -100_000, 100_000)
              # si es un nodo mejor que el actual
              if (maximize and best_v < val) or (not maximize and best_v > val): 
                  best_v, best_m = val, m
                  self.d[hash] = BestMove(best_m, best_v, self.depth)
              board.pop()
          
        return self.d[hash]

    def solve_position(self, fen, max_d):
        self.depth = max_d
        return self.get_move(chess.Board(fen))

    # get_random_board -> devuelve una posición aleatoria de un tablero de ajedrez
    def get_random_board(self, n_moves=100):
        board = chess.Board()
        while n_moves > 0 and not board.is_game_over():
            choosen = random.choice(list(board.legal_moves))
            board.push(choosen)
            if not board.is_valid():
                board.pop()
            n_moves -= 1
        return board
    

In [489]:
import time
t = time.time()
eng = Engine()
fen = '8/5p2/7k/p1p3Rp/1pPr1q2/1P1P3R/P3r3/1K6 w - - 0 49'
bm = eng.solve_position(fen,2)

In [492]:
print(bm)
print(len(eng.d.keys()))
print(time.time()-t)

for k, v in eng.d.items():
  print(k,v)

g5h7
182
28.077670574188232
2432471717103137135 h3h5 - -5 - 1
17529511519621675630 h3h5 - -9 - 1
17620632951513503419 h3h5 - -9 - 1
10104326817334019315 h3h5 - 100000 - 2
9524819498519794223 h5c5 - -4 - 1
10427645645944079304 h5c5 - -4 - 1
13944333829932831509 f5f1 - -1 - 1
15442562147136802212 b1c1 - -1 - 1
13347408782826915274 b1b2 - -2 - 1
6638260890395011159 b1c2 - -6 - 1
5833568307785463705 f5f7 - -5 - 1
7849167157610506296 f5f7 - -5 - 1
8679935496275671519 f5f6 - -6 - 1
2658977475228399361 f5f6 - -6 - 1
1955693240521602076 f5f6 - -6 - 1
12186141975344861505 f5f6 - -6 - 1
17683782050633706120 f5f6 - -6 - 1
8801762093544623348 h3h5 - -9 - 1
3263914690971472367 f5f6 - -6 - 1
3705865091389211868 f5f6 - -6 - 1
2817129601691745237 f5f6 - -6 - 1
13028295129462624129 f5f6 - -6 - 1
15876316337158463034 f5f6 - -6 - 1
12698963573318653339 f5f6 - -6 - 1
2363901808756121979 f5f6 - -6 - 1
18337820287905754961 f5f6 - -6 - 1
7500829473085562908 f5f6 - -6 - 1
3844480231240606235 f5f6 - -6 - 1
339

In [493]:
e = Engine(1)
b = chess.Board()
cont = 5
while cont > 0 and b.is_valid() and not b.is_game_over():
  bm = e.get_move(b).move
  b.push(bm)
  cont -= 1

for k, v in e.d.items():
  print(k,v)

17799680033381230760 g1h3 - 0 - 1
8631851012748312859 h3g5 - 0 - 1
15934100002695402808 g8h6 - 0 - 1
106468305598532662 h3g5 - 0 - 1
8481853096639713680 h3g5 - 0 - 1
9734674445742320494 h3g5 - 0 - 1
7112112334875565656 h3g5 - 0 - 1
14034000663834222067 h3g5 - 0 - 1
6189532799725940978 h3g5 - 0 - 1
5469452786960054884 h3g5 - 0 - 1
15793581086578919671 h3g5 - 0 - 1
6451064739526070548 h3g5 - 0 - 1
15400717762136273079 h3g5 - 0 - 1
16516195293455731487 h3g5 - 0 - 1
8087651837776138762 h3g5 - 0 - 1
9754172516212021389 h3g5 - 1 - 1
10073560245779014628 h3g5 - 0 - 1
13333225990945745710 h3g5 - 0 - 1
9427080393920447644 h3g5 - 0 - 1
1230503383515884598 h3g5 - 0 - 1
7218395560896877374 h3g5 - 0 - 1
10321649396316913897 h3g5 - 0 - 1
9956593560585119264 g5h7 - 1 - 1
8300396683082523700 h8g8 - 1 - 1
15826842310869704860 g5h7 - 1 - 1
2956938185999391330 g5h7 - 1 - 1
15698382315526205463 g5h7 - 1 - 1
1588779896120848033 g5h7 - 1 - 1
12316228194151043873 g5h7 - 1 - 1
7841730632268672255 g5h7 - 1 - 1

In [494]:
e4 = Engine(4)
b = chess.Board()
cont = 5
while cont > 0 and b.is_valid() and not b.is_game_over():
  bm = e4.get_move(b).move
  b.push(bm)
  cont -= 1

for k, v in e4.d.items():
  print(v)

KeyboardInterrupt: ignored